In [ ]:
## MYSQL 관련 모듈 설치
 
- conda install MySQL-python  or
- pip install MySQL-python


- wget http://dev.mysql.com/get/Downloads/Connector-Python/mysql-connector-python-cext_2.1.3-1ubuntu14.04_amd64.deb
- sudo dpkg -i  mysql-connector-python-*.deb
- or
- conda install mysql-connector-python


In [ ]:
# MySQL-python 을 이용할때

import MySQLdb as mdb
import sys

try:
    con = mdb.connect('node02', 'deepbio', 'deepbio1234', 'deepbioDB');
     
    cur = con.cursor()
    cur.execute("SELECT VERSION()")

    ver = cur.fetchone()
    
    print "Database version : %s " % ver
    
except mdb.Error, e:
  
    print "Error %d: %s" % (e.args[0],e.args[1])
    sys.exit(1)
    
finally:    
        
    if con:    
        con.close()

In [ ]:
# mysql-connector-python을 이용할때

import mysql.connector as mariadb

try:
    mariadb_connection = mariadb.connect(host='node02', user='deepbio', password='deepbio1234', database='deepbioDB')
    cursor = mariadb_connection.cursor()
    cursor.execute("SELECT VERSION()")
    ver = cursor.fetchone()
    print "Database version : %s " % ver
    
except mariadb.Error as error:
    print("Error: {}".format(error))
    
mariadb_connection.commit()

mariadb_connection.close()    

### DB 와 HBase 스키마 정의
```
-- ##############################################################
-- DB 스키마 정의
-- ##############################################################

CREATE SCHEMA IF NOT EXISTS `deepbioDB` DEFAULT CHARACTER SET utf8 COLLATE utf8_general_ci ;
USE `deepbioDB` ;

DROP TABLE IF EXISTS file_manifest ;

CREATE TABLE IF NOT EXISTS file_manifest (
  `id`       MEDIUMINT NOT NULL AUTO_INCREMENT,
  `disease` VARCHAR(10) NOT NULL,
  `platform_type` VARCHAR(50) NOT NULL,
  `center` VARCHAR(10) NOT NULL,
  `platform` VARCHAR(50) NOT NULL,
  `level` VARCHAR(10) NOT NULL,
  `barcode` VARCHAR(50) NOT NULL,
  `file_name` VARCHAR(512) NOT NULL,
  `reg_dt` TIMESTAMP DEFAULT CURRENT_TIMESTAMP  ,
  `flag`       MEDIUMINT DEFAULT 0 ,
  PRIMARY KEY (`id`),
  index file_manifefile_annotationsst_barcode_idx(barcode), 
  index file_manifefile_annotationsst_disease_idx(disease) 
)
ENGINE = InnoDB;


select count(*) from file_manifest;
select * from file_manifest limit 10;
select count(*) from file_annotations;

DROP TABLE IF EXISTS file_annotations ;

CREATE TABLE IF NOT EXISTS file_annotations (
  `id`       MEDIUMINT NOT NULL ,
  `disease` VARCHAR(10) NOT NULL,
  `item_type` VARCHAR(50) NOT NULL,
  `item_barcode` VARCHAR(50) NOT NULL,
  `item_uuid` VARCHAR(50) NOT NULL,
  `classification` VARCHAR(10) NOT NULL,
  `category` VARCHAR(50) NOT NULL,
  `annotator` VARCHAR(50) NOT NULL,
  `date_created` VARCHAR(20) NOT NULL,
  `annotation` VARCHAR(4096) NOT NULL,
  `reg_dt` TIMESTAMP DEFAULT CURRENT_TIMESTAMP  ,
  PRIMARY KEY (`id`),
  index file_annotations_disease_idx(disease) ,  
  index file_annotations_barcode_idx(item_barcode) ,
  index file_annotations_uuid_idx(item_uuid) ,
  index file_annotations_date_created_idx(date_created) 
)
ENGINE = InnoDB;


-- ##############################################################
-- HBASE 스키마 정의
-- ##############################################################

disable 'mRNA'
drop 'mRNA'
create 'mRNA', {NAME => 'manifest' },  {NAME=>'data', COMPRESSION=>'SNAPPY'}


```

## CSV와 Tab을 구분된 파일 읽어서 DB에 넣는 모듈 

- 다른 장비에 있는 파일을 가지고 오기.

```
scp  spark@node03:/home/spark/tcga/01/file_annotations.txt  /home/deepbio/note/work/tcga/node03_01_file_annotations.txt
scp  spark@node03:/home/spark/tcga/02/file_annotations.txt  /home/deepbio/note/work/tcga/node03_02_file_annotations.txt
scp  spark@node03:/home/spark/tcga/03/file_annotations.txt  /home/deepbio/note/work/tcga/node03_03_file_annotations.txt

scp  spark@node03:/home/spark/tcga/01/file_manifest.txt /home/deepbio/note/work/tcga/node03_01_file_manifest.txt
scp  spark@node03:/home/spark/tcga/02/file_manifest.txt /home/deepbio/note/work/tcga/node03_02_file_manifest.txt
scp  spark@node03:/home/spark/tcga/03/file_manifest.txt /home/deepbio/note/work/tcga/node03_03_file_manifest.txt


scp  spark@node04:/home/spark/tcga/01/file_annotations.txt  /home/deepbio/note/work/tcga/node04_01_file_annotations.txt
scp  spark@node04:/home/spark/tcga/02/file_annotations.txt  /home/deepbio/note/work/tcga/node04_02_file_annotations.txt
scp  spark@node04:/home/spark/tcga/03/file_annotations.txt  /home/deepbio/note/work/tcga/node04_03_file_annotations.txt
scp  spark@node04:/home/spark/tcga/04/file_annotations.txt  /home/deepbio/note/work/tcga/node04_04_file_annotations.txt
scp  spark@node04:/home/spark/tcga/05/file_annotations.txt  /home/deepbio/note/work/tcga/node04_05_file_annotations.txt
scp  spark@node04:/home/spark/tcga/01/file_manifest.txt /home/deepbio/note/work/tcga/node04_01_file_manifest.txt
scp  spark@node04:/home/spark/tcga/02/file_manifest.txt /home/deepbio/note/work/tcga/node04_02_file_manifest.txt
scp  spark@node04:/home/spark/tcga/03/file_manifest.txt /home/deepbio/note/work/tcga/node04_03_file_manifest.txt
scp  spark@node04:/home/spark/tcga/04/file_manifest.txt /home/deepbio/note/work/tcga/node04_04_file_manifest.txt
scp  spark@node04:/home/spark/tcga/05/file_manifest.txt /home/deepbio/note/work/tcga/node04_05_file_manifest.txt

```

In [ ]:
! ls -al /home/deepbio/note/work/tcga/

## mRNA 데이터파일을 HBASE에 넣는 모듈 

In [43]:
##
## file_manifest.txt 파일과 file_annotations.txt 파일을 DB에 upload하는 모듈
## upload_db.py

DB_HOST = 'node02'
DB_USER = 'deepbio'
DB_PW = 'deepbio1234'
DB_NAME = 'deepbioDB'
BULK_INSERT_SIZE = 500

SEPARATOR = '\t'
INT_COLUMNS = ['id']


# 해더를 파싱해서 DB columns 리스트로 변경함.
# 모두 소문자 처리, 공백은 언더바로 변경함.
def divHeader( line,  sep = '\t') :
    if None == line : return None

    cols = line.split( sep )
    for idx in range(len(cols)):
        cols[idx] = cols[idx].lower().replace(' ', '_')
    return cols

#   INSERT 문을 만들어주는 함수
#  예) INSERT INTO tablename ( col1, col2, col3, col4 )  values ( 'val1', 'val2', 'val3' )
def makeInsertSQL( tablename, columns, rows ) :    
    if  len( columns ) >  len( rows )   :  return None
    
    from cStringIO import StringIO
    insertSQL = StringIO()
    insertSQL.write('INSERT INTO ')
    insertSQL.write( tablename )
    insertSQL.write(' ( ')
    for idx in range(len(columns)) : 
        insertSQL.write( columns[idx] )
        if( len(columns)-1 > idx ) : insertSQL.write( ', ' )
    insertSQL.write(' ) ')

    insertSQL.write(' VALUES ( ')
    for idx in range(len(columns)) : 
        if columns[idx] not in  INT_COLUMNS : insertSQL.write("'")    
        insertSQL.write( rows[idx] )
        if columns[idx] not in  INT_COLUMNS : insertSQL.write("'")
        if( len(columns)-1 > idx ) : insertSQL.write( ', ' )
    insertSQL.write(' ) ')    
    #print(insertSQL.getvalue())
    return insertSQL.getvalue()    

# 
# 데이터 파일을 읽어서 DB에 저장하는 함수.
#
def fileReadAndInsert( dbconnection, cursor , fileName, tablename ) : 
    headers = []
    rows = []
    isHeader = True
    sqlCount = 0
    commitCount = 0    

    fhand = open(  fileName  )
    for line in fhand:
        if True == isHeader : 
            headers = divHeader( line, SEPARATOR )
            isHeader = False
            continue

        rows = line.strip().split( SEPARATOR )
        
        #print 'headers', len( headers )
        #print ', rows', len( rows )        
        #break
        #if  len( headers ) >  len( rows )   : 
        #    print line
        #    continue
        
        sql = makeInsertSQL(tablename, headers, rows )
        try:
            if  None !=  cursor and  None != sql  :  cursor.execute( sql )
        except mariadb.Error as error:
            print("Error: {}".format(error))

        sqlCount += 1
        if sqlCount >= BULK_INSERT_SIZE : 
            if  None !=  dbconnection : dbconnection.commit()
            commitCount += sqlCount
            print "commit count=", commitCount
            sqlCount = 0

    if  None !=  dbconnection : dbconnection.commit()            
    commitCount += sqlCount
    print "commit count=", commitCount    
    return commitCount


import mysql.connector as mariadb
mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)
cursor = mariadb_connection.cursor()
cursor.execute('SET autocommit = 0')

#mariadb_connection = None
#cursor = None

TABLE_NAME = 'file_manifest'
DATA_FILES = ['/home/deepbio/note/work/tcga/node03_01_file_manifest.txt'
             ,'/home/deepbio/note/work/tcga/node03_02_file_manifest.txt'
             ,'/home/deepbio/note/work/tcga/node03_03_file_manifest.txt'
             ,'/home/deepbio/note/work/tcga/node04_01_file_manifest.txt'
             ,'/home/deepbio/note/work/tcga/node04_02_file_manifest.txt'
             ,'/home/deepbio/note/work/tcga/node04_03_file_manifest.txt'
             ,'/home/deepbio/note/work/tcga/node04_04_file_manifest.txt' 
             ,'/home/deepbio/note/work/tcga/node04_05_file_manifest.txt']

#TABLE_NAME = 'file_annotations'
#DATA_FILES = [ '/home/deepbio/note/work/tcga/node03_01_file_annotations.txt'
#             ,'/home/deepbio/note/work/tcga/node03_02_file_annotations.txt'
#             ,'/home/deepbio/note/work/tcga/node03_03_file_annotations.txt'
#             ,'/home/deepbio/note/work/tcga/node04_01_file_annotations.txt'
#             ,'/home/deepbio/note/work/tcga/node04_02_file_annotations.txt'
#             ,'/home/deepbio/note/work/tcga/node04_03_file_annotations.txt'
#             ,'/home/deepbio/note/work/tcga/node04_04_file_annotations.txt'
#             ,'/home/deepbio/note/work/tcga/node04_05_file_annotations.txt']


#TABLE_NAME = 'file_manifest'
#DATA_FILES = [ '/home/deepbio/note/work/tcga/node03_01_file_manifest.txt'
#             ,'/home/deepbio/note/work/tcga/node03_02_file_manifest.txt'
#             ,'/home/deepbio/note/work/tcga/node03_03_file_manifest.txt' ]


totalCommit = 0
for datafile in  DATA_FILES : ㅡ
    totalCommit += fileReadAndInsert( mariadb_connection, cursor , datafile, TABLE_NAME )

if  None !=  mariadb_connection : mariadb_connection.commit()
if  None !=  mariadb_connection : mariadb_connection.close()

print "Total INSERT=", totalCommit

commit count= 500
commit count= 1000
commit count= 1500
commit count= 2000
commit count= 2500
commit count= 3000
commit count= 3500
commit count= 4000
commit count= 4500
commit count= 5000
commit count= 5500
commit count= 6000
commit count= 6500
commit count= 7000
commit count= 7500
commit count= 8000
commit count= 8500
commit count= 9000
commit count= 9500
commit count= 10000
commit count= 10500
commit count= 11000
commit count= 11500
commit count= 11516
commit count= 500
commit count= 1000
commit count= 1500
commit count= 2000
commit count= 2500
commit count= 3000
commit count= 3500
commit count= 4000
commit count= 4500
commit count= 5000
commit count= 5500
commit count= 6000
commit count= 6500
commit count= 7000
commit count= 7500
commit count= 8000
commit count= 8500
commit count= 9000
commit count= 9500
commit count= 10000
commit count= 10500
commit count= 11000
commit count= 11500
commit count= 12000
commit count= 12500
commit count= 12736
commit count= 500
commit count= 1000
com

- 설치 방법
```
(envname) $ pip install happybase
```

- hbase는 thrift server가 구동이 되어야함.

```
bin/hbase-daemon.sh start thrift  
```


In [25]:
# 접속 확인
import happybase
connection = happybase.Connection('node03')
table = connection.table('mRNA')
print(table)

<happybase.table.Table name='mRNA'>


In [ ]:
#-*- coding: utf-8 -*-
#
# DB에 등록된 정보를 이용해서 물리적인 파일을 HBASE에 올리는 모듈 
# upload_hbase.py

DB_HOST = 'node02'
DB_USER = 'deepbio'
DB_PW = 'deepbio1234'
DB_NAME = 'deepbioDB'
QUERY_FETCH_SIZE = 100
BASE_PATH = '/home/deepbio/note/work/tcga/01/'
DISEASES  = [ 'COAD'
             ,'DLBC'
             ,'ESCA'
             ,'FPPP'
             ,'GBM'
            ]

## Disease Platform Type   Center  Platform        Level   Barcode File Name
## COAD    RNASeqV2        UNC     IlluminaGA_RNASeqV2     Level_3 TCGA-AA-3814-01A-01R-0905-07    unc.edu.51364582-19a3-4d49-8a15-2f7f42860e39.1589076.rsem.isoforms.results
## => /COAD/RNASeqV2/UNC__IlluminaHiSeq_RNASeqV2/Level_3
def processDBData( cursor, pool,cursor_2 ) :
    #mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)
    #cursor_3 = mariadb_connection.cursor()
    #cursor_3.execute('SET autocommit = 1')
    print("now run")
    fullPath = ''
    for ID ,disease, platform_type, center, platform, level, barcode, file_name  in cursor:
        fullPath =  "{}/{}/{}__{}/{}/{}".format(disease, platform_type, center, platform, level, file_name) 
        #print(fullPath)
        contents = readFile( BASE_PATH + fullPath ) 
        if None == contents : continue 
        uploadFileToHBase( pool, disease, platform_type, center, platform, level, barcode, file_name, contents )
        sql = "UPDATE file_manifest set flag = 1 where id = '%d'" % ID 
        #cursor_3.execute("UPDATE file_manifest set flag = 1 where id = %d " % ID)
        cursor_2.execute(sql)
        
import os 
##  파일의 전체 내용을 String으로 읽어들임.
def readFile( fname ) :
    if True != os.path.exists(fname) : 
        print 'read not ' + fname
        return None

    content = None
    with open(fname, 'r') as content_file:
        content = content_file.read()

    return content    
    
## HBASE로 데이터를 업로드함.
def uploadFileToHBase( pool, disease, platform_type, center, platform, level, barcode, file_name,  contents ) :
    with pool.connection() as connection:
        table = connection.table('mRNA')
        #print 'put rowkey=' + file_name
        table.put(file_name, {'manifest:disease': disease,
                       'manifest:platform_type': platform_type,
                       'manifest:center': center,
                       'manifest:platform': platform,
                       'manifest:level': level,
                       'manifest:barcode': barcode,
                       'manifest:file_name': file_name,
                       'data:contents': contents })
        
        
import happybase
pool = happybase.ConnectionPool(size=3, host='node03')    
    
import mysql.connector as mariadb
mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)
cursor = mariadb_connection.cursor()

mariadb2_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)
cursor2 = mariadb2_connection.cursor()
cursor2.execute('SET autocommit = 1')


whereDisease = ''
for disease in DISEASES :
    whereDisease +=  ("'" + disease + "' ,")

whereDisease = whereDisease[ : -1]
    
cursor.execute("SELECT count(*) as totalcount FROM file_manifest WHERE flag = 0 AND disease in (%s) " % whereDisease )
totalCount = cursor.fetchone() 
print "totalCount=", totalCount

offset = 0;
pageCount = totalCount[0] / QUERY_FETCH_SIZE 
if totalCount[0] % QUERY_FETCH_SIZE  > 0 : 
    pageCount += 1

# debug용
#pageCount = 1
    
for idx in range( pageCount )  :
    offset = idx * QUERY_FETCH_SIZE
    cursor.execute("SELECT id ,disease, platform_type, center, platform, level, barcode, file_name FROM file_manifest WHERE flag = 0 AND disease in (%s) LIMIT %d, %d" %(whereDisease, offset,QUERY_FETCH_SIZE))
    processDBData( cursor, pool,cursor2 )
        
    
mariadb_connection.close()
mariadb2_connection.close()